<a href="https://colab.research.google.com/github/Ankitha2130/Ankitha2130/blob/main/Skin_disease_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
shubhamgoel27_dermnet_path = kagglehub.dataset_download('shubhamgoel27/dermnet')

print('Data source import complete.')


100%|██████████| 1.72G/1.72G [00:19<00:00, 93.6MB/s]

Extracting files...


Data source import complete.


In [2]:
import numpy as np
import cv2
import random
import os
import matplotlib.pyplot as plt

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [3]:
data_path = '/kaggle/input/dermnet/train'

In [4]:
train_data = []
test_data = []

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    file = os.listdir(folder_path)
    num_train = int(0.6 * len(file))
    files_train = random.sample(file, num_train)
    files_test = list(set(file) - set(files_train))

    for file in files_train:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        train_data.append((img, folder))

    for file in files_test:
        file_path = os.path.join(folder_path, file)
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224,224))
        test_data.append((img, folder))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/dermnet/train'

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 5))
plt.suptitle('LABELS OF THE IMAGE')

for (img, label), ax in zip(random.sample(train_data, 8), axes.flatten()):
    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.grid(True)
    ax.set_title(label)
    ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

plt.show()

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
base_model.trainable = False

In [ ]:
classes = 23
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(classes, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X_train, y_train = zip(*train_data)
X_test, y_test = zip(*test_data)

X_train = preprocess_input(np.array(X_train))
X_test = preprocess_input(np.array(X_test))

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

y_train_one_hot = to_categorical(y_train_encoded, classes)
y_test_one_hot = to_categorical(y_test_encoded, classes)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
custom_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    mode='min'
)

In [ ]:
EPOCHS = 20
BATCH_SIZE = 64
history = model.fit(X_train, y_train_one_hot, validation_data=(X_test, y_test_one_hot),
                   epochs = EPOCHS, batch_size=BATCH_SIZE,callbacks=[custom_early_stopping])

model.save('/kaggle/working/my_model.h5')